In [1]:
from transformers import BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoModelForMaskedLM

tokenizer = BertTokenizer.from_pretrained("symbol-vocab")
mask_model = AutoModelForMaskedLM.from_pretrained("./symbol-masked-trainer/checkpoint-50000")
model = AutoModelForSequenceClassification.from_pretrained("./symbol-masked-trainer/checkpoint-50000", num_labels=2)
# model.bert = mask_model

OSError: ./symbol-masked-trainer/checkpoint-50000 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [2]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

def data_loading(url):
    with open(url, 'r', encoding='utf-8') as f:
        # data = json.loads(f.read())
        df = pd.read_json(f)
        data = df.copy()
        # data = df.loc[:, ['sentiment', 'body']]
        data = data.loc[df['sentiment'].notnull()]
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['label'] = data['sentiment'].cat.codes
        data = data.rename(columns={'sentiment': 'labels', 'body': 'sentense'})

        symbols = set()
        for symbol_list in data['sentense'].str.findall(r'\$[A-Z]+'):
            for symbol in symbol_list:
                symbols.add(symbol)
        return data, symbols

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data, symbols = data_loading(url)

dataset = Dataset.from_pandas(data.loc[:, ['label', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

# special_tokens_dict = {'additional_special_tokens': list(symbols)}
# tokenizer.add_special_tokens(special_tokens_dict)
# model.resize_token_embeddings(len(tokenizer))

In [3]:
from parse import normalize_except_compony

def encode(example):
    sentense = normalize_except_compony(example['sentense'])

    result = tokenizer(
        sentense,
        padding='max_length',
        truncation=True,
    )
    return result

encoded_dataset = dataset.map(encode, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)
# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# type(encoded_dataset[0]['labels'])

100%|██████████| 15/15 [00:10<00:00,  1.40ba/s]


In [4]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

import numpy as np
import wandb

%env WANDB_PROJECT=Stocktwit_sentiment_analysis
training_args = TrainingArguments(
    output_dir="bert-classifier-trainer",
    evaluation_strategy="steps",
    eval_steps=100,
    num_train_epochs=20,
    save_steps=5000,
    per_device_train_batch_size=4,
    report_to="wandb",
    run_name="fin-bert-sentiment-classifier-20-epoch"
)

recall_metric = load_metric("recall")
precision_metric = load_metric("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    result = recall_metric.compute(predictions=predictions, references=labels)
    result.update(precision_metric.compute(predictions=predictions, references=labels))
    return result

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
)

trainer.train()

wandb.finish()

env: WANDB_PROJECT=Stocktwit_sentiment_analysis


The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\douli\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11353
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 56780
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it 

  0%|          | 100/56780 [00:30<4:33:08,  3.46it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2839
  Batch size = 8

  0%|          | 100/56780 [01:38<4:33:08,  3.46it/s]

{'eval_loss': 0.6357525587081909, 'eval_recall': 1.0, 'eval_precision': 0.6660796054948925, 'eval_runtime': 67.5249, 'eval_samples_per_second': 42.044, 'eval_steps_per_second': 5.257, 'epoch': 0.04}


  0%|          | 200/56780 [02:06<4:23:11,  3.58it/s]  The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2839
  Batch size = 8

  0%|          | 200/56780 [03:10<4:23:11,  3.58it/s]

...

100%|█████████▉| 56700/56780 [14:22:33<00:22,  3.64it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2839
  Batch size = 8
                                                        
100%|█████████▉| 56700/56780 [14:23:36<00:22,  3.64it/s]

{'eval_loss': 0.6307917833328247, 'eval_recall': 0.9740877842411423, 'eval_precision': 0.6834879406307978, 'eval_runtime': 62.5514, 'eval_samples_per_second': 45.387, 'eval_steps_per_second': 5.675, 'epoch': 19.97}


100%|█████████▉| 56779/56780 [14:23:57<00:00,  3.63it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 56780/56780 [14:23:57<00:00,  1.10it/s]


{'train_runtime': 51841.6593, 'train_samples_per_second': 4.38, 'train_steps_per_second': 1.095, 'train_loss': 0.6198767188738332, 'epoch': 20.0}



eval/loss,▄█▁▁▄▁▅▆▆▇▃▃▄▃▇▄▅▅▅▅▅▆▅▄▄▆▅▆▅▆▄▅▅▅▆▅▅▅▅▅
eval/precision,▂▁█▄▅▄▂▁▁▂▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
eval/recall,██▁▆▅▆███▇▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
eval/runtime,█▄▃▃▃▃▂▂▂▃▃▂▂▂▂▅▅▅▆▅▅▆▅▅▅▅▅▄▅▄▅▅▅▅▄▅▁▁▁▂
eval/samples_per_second,▁▅▆▆▆▆▆▇▇▆▆▇▇▇▇▄▄▄▃▄▄▃▄▄▄▄▄▄▄▅▄▄▄▄▄▄███▇
eval/steps_per_second,▁▅▆▆▆▆▆▇▇▆▆▇▇▇▇▄▄▄▃▄▄▃▄▄▄▄▄▄▄▅▄▄▄▄▅▄███▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▄▁▄▃▅▅▅▄▄▃▂▅▄▄▅▅▅▅▄▅▅▆▅▅▄▅▅▅▅▅▅▄▄▅▄▅▅▄
train/total_flos,▁
